In [4]:
import matplotlib.pyplot as plt
from PIL import Image, ImageFont, ImageDraw
import numpy as np
import os, glob
import imageio  

In [5]:
import csv

f = open("/home/jeongmo/AMD_data/ID.csv",'w',newline="")
wr = csv.writer(f)

ID_w = []
trial_w = []

In [6]:
OCT_folder = '/home/jeongmo/AMD_data/OCT_result'  # 결과를 도출할 폴더
source_folder = '/home/jeongmo/AMD_data/result_2022'   # source file folder 를 다른 /dev/sdb2 등은 어떻게 불러오나? terminal에서 /dev/로 옮기기  
bscan_number = [13,14,15,16,17,18]
# type(str(bscan_number[0]))
title_font = ImageFont.truetype(font='/home/jeongmo/AMD_data/tahomabd.ttf', size=20)

currentpath = os.getcwd() #현재의 working directory 보는 것
datapath = source_folder  #  "/home/jeongmo/AMD_data/Extract_2022"      #/media/oph/octgan/dataset/data/extracted
os.chdir(datapath)   #working directory 변경

OCT_size_error = []

ID = glob.glob('*')
ID.sort()
for i in ID:
    img_pixel = []

    ID_path = datapath + '/' + i
    os.chdir(ID_path)
    trial = glob.glob ('*')
    # trial.sort()
    trial = sorted(trial, key=lambda x:int(x.split("(")[0].split("l")[1]))
    # 이렇게 sort를 하지 않으면 trial 0, 1, 10, 2, 3 ... 순으로 배열됨

    
    
    for j in trial:
        trial_path = ID_path + '/' + j
        os.chdir(trial_path)
        # print(trial_path)
        img_png = Image.open('slo.png')   #os.chdir 로 되어 있으나 아니 될 경우에는 trial_paht + slo.png 로 변경 고려

        if not img_png.size == (768,768):
            OCT_size_error.append({img_png.size, i, j, "PNG_size_error"})
            continue  # break 로 하면 안됨

        editable = ImageDraw.Draw(img_png)
        title_text = trial_path.split('/')[-1]
        editable.text((570,740), title_text, (255), font=title_font)  # text 위치
              

        img_pixel = np.array(img_png)
    
    

        for l in bscan_number:
            bscan_name = f'bscan_{l}.png'
            
            img_OCT = Image.open(bscan_name)

            
            if not img_OCT.size == (768,496):
                OCT_size_error.append({img_OCT.size, i, j, l})
        
                continue  # break 로 하면 안됨



            

            editable = ImageDraw.Draw(img_OCT)
            title_text1 = i
            title_text2 = f'bscan_{l}'

            editable.text((600,440), title_text1, (255), font=title_font)  # text 위치
            editable.text((650,460), title_text2, (255), font=title_font)  # text 위치


            img_OCT_pixel = np.array(img_OCT)
            
            img_pixel = np.vstack ((img_pixel,img_OCT_pixel))
        
        if not img_OCT.size == (768,496):
            continue  # break 로 하면 안됨

        if j == trial[0]:
            img_pixel_sum = img_pixel
        else:
     
            img_pixel_sum = np.hstack ((img_pixel_sum,img_pixel))
     
    img_sum_png = Image.fromarray(img_pixel_sum)
    
    OCT_name = OCT_folder+'/' + i + '.jpg'                    #f"{SLO_folder}/{i}.png"
 
    wr.writerow ([ID_w,trial_w])
    
    img_sum_png.save(OCT_name)

f.close()



